In [1]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

def resposta_simples (query):
    return f"A resposta para '{query}' é Paris."



C:\Users\arthu\AppData\Local\Temp\ipykernel_28412\2422021406.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [2]:
tools = [
    Tool(
        name="Respostas Simples",
        func=resposta_simples,
        description="Uma ferramenta simples para responder perguntas diretas"
    )
]

In [3]:
agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

input_query = "Qual é a capital da frança?"
response = agent.run(input_query)
print(response)

C:\Users\arthu\AppData\Local\Temp\ipykernel_28412\4125823330.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
C:\Users\arthu\AppData\Local\Temp\ipykernel_28412\4125823330.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth



> Entering new AgentExecutor chain...
 I should use a tool to find the answer
Action: Respostas Simples
Action Input: Qual é a capital da frança?
Observation: A resposta para 'Qual é a capital da frança?' é Paris.
Thought: I now know the final answer
Final Answer: Paris

> Finished chain.
Paris


In [10]:
from langchain.agents import AgentExecutor
import sqlite3

In [11]:
def create_database():
    conn = sqlite3.connect(":memory:")
    c = conn.cursor()
    c.execute("""CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)""")
    c.execute("INSERT INTO users (name, age) VALUES('joão', 25), ('Maria', 30)")
    conn.commit()
    return conn

def search_user_db(user_name):
    conn = create_database()
    c = conn.cursor()
    c.execute("SELECT name, age FROM users WHERE name = ?", (user_name, ))
    result = c.fetchone()
    conn.close()
    if result:
        return f"{result[0]}, tem {result[1]} anos"
    else:
        return "Usuario não encontrado"
    
def sum_numbers(numbers):
    if isinstance(numbers, str):
        numbers = eval(numbers)

    if isinstance(numbers, list):
        total = sum(numbers)
        return f"A soma dos {numbers}, é {total} "
    
    else:
        return "erro. a entrada não é um alista de numeros validos"

In [12]:
tools = [
    Tool(
        name="Buscar Usuário no DB",
        func= search_user_db,
        description= "Procura informações sobre um usuário na base de dados"
    ),
    Tool(
        name= "Calcular Soma",
        func= sum_numbers,
        description= "Calcula a soma de uma lista de numeros"
    )
]

In [13]:
agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

input_query = "Qual é a soma de 5, 10 e 15? Quantos anos Maria tem?"
response = agent.run(input_query)
print(response)



> Entering new AgentExecutor chain...
 I need to calculate the sum of the numbers and search for Maria's age in the database.
Action: Calcular Soma
Action Input: 5, 10, 15
Observation: erro. a entrada não é um alista de numeros validos
Thought: I need to input a list of numbers for the calculation.
Action: Calcular Soma
Action Input: [5, 10, 15]
Observation: A soma dos [5, 10, 15], é 30 
Thought: Now I need to search for Maria's age in the database.
Action: Buscar Usuário no DB
Action Input: Maria
Observation: Maria, tem 30 anos
Thought: I now know the final answer.
Final Answer: A soma dos números é 30 e Maria tem 30 anos.

> Finished chain.
A soma dos números é 30 e Maria tem 30 anos.
